# Лабораторная работа №2

Описание

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

Описание данных

- airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
- balcony — число балконов
- ceiling_height — высота потолков (м)
- cityCenters_nearest — расстояние до центра города (м)
- days_exposition — сколько дней было размещено объявление (от публикации до снятия)
- first_day_exposition — дата публикации
- floor — этаж
- floors_total — всего этажей в доме
- is_apartment — апартаменты (булев тип)
- kitchen_area — площадь кухни в квадратных метрах (м²)
- last_price — цена на момент снятия с публикации
- living_area — жилая площадь в квадратных метрах (м²)
- locality_name — название населённого пункта
- open_plan — свободная планировка (булев тип)
- parks_around3000 — число парков в радиусе 3 км
- parks_nearest — расстояние до ближайшего парка (м)
- ponds_around3000 — число водоёмов в радиусе 3 км
- ponds_nearest — расстояние до ближайшего водоёма (м)
- rooms — число комнат
- studio — квартира-студия (булев тип)
- total_area — площадь квартиры в квадратных метрах (м²)
- total_images — число фотографий квартиры в объявлении

Для начала импортируем все библиотеки и считываем файл с датасетом. При этом сталкиваемся с проблемой - pandas всё видит как одну колонку. В нашем случае разделители у датасета не стандартная запятая, а табуляция, поэтому чтобы pandas считал всё правильно мы дописываем параметр sep='\t' при чтении файла.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('real_estate_data.csv', sep='\t')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

### Обработка пропусков

Можно заметить, что в датасете есть пропуски в столбцах: ceiling_height, living_area, is_apartment, kitchen_area, balcony, airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition. 

Для некоторых пропущенных значений можно предположить логичную замену. Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски допустимо заменить на 0.

In [2]:
data['balcony'] = data['balcony'].fillna(0)
# data['parks_around3000'] = data['parks_around3000'].fillna(0)
data['ponds_around3000'] = data['ponds_around3000'].fillna(0)

Теперь проанализируем информацию о близлежащих объектах. 

Аналогичным способом стоит заменить пропуски в ponds_around3000, более того, по итогу, во всех строках , где ponds_around3000 равен 0, столбец ponds_nearest содержит пропуск, что логично. Естественно, в ponds_nearest пропуски заменять на что либо нет смысла, т.к. расстояния до пруда буквально нет, если его рядом нет.

In [ ]:
data['ponds_around3000'].value_counts(dropna=False)

In [ ]:
data[(data['ponds_around3000'] == 0) & (data['ponds_nearest'].isna())]

In [46]:
data['ponds_nearest'].min()

np.float64(13.0)

Также мы поступим и с парками. Единственное отличие между парками и прудами в том, что у парков количество нулей и NaN не совпадает у колонок — у parks_nearest меньше "NaN", чем нулей у parks_around3000 на 4. Произошло это из-за того, что у parks_nearest есть 4 значения, которые больше 3000 метров, поэтому они соответствуют нулю в столбце parks_around3000.

In [57]:
mask = data['parks_nearest'].isna()  # строки, где parks_nearest == NaN
data.loc[mask, 'parks_around3000'] = data.loc[mask, 'parks_around3000'].fillna(0)

In [ ]:
data['parks_nearest'].value_counts(dropna=False)

In [ ]:
data['parks_around3000'].value_counts(dropna=False)

In [ ]:
data[data['parks_nearest'] > 3000]

У столбцов ceiling_height, living_area, kitchen_area, days_exposition никак заполнить пропуски не выйдет, т.к. не на что ориентироваться при выборе значений.

In [ ]:
data['floors_total'].value_counts(dropna=False)

np.float64(1.0)

Со столбцом is_apartment всё страннее, но пусть будет пока так:

In [97]:
data['is_apartment'] = data['is_apartment'].fillna(False)

C:\Users\mystr\AppData\Local\Temp\ipykernel_21904\2052809825.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['is_apartment'] = data['is_apartment'].fillna(False)


In [99]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          23699 non-null  bool   
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               23699 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   